Load all required modules

In [1]:
# Load required modules
import pandas as pd
import numpy as np

# For maps
!pip -q install folium
import folium
!pip -q install geopy
from geopy.geocoders import Nominatim

# For APIs
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


# For plots
import matplotlib.cm as cm
import matplotlib.colors as colors

# For clustering
from sklearn.cluster import KMeans
print("Libraries ready")


Libraries ready


Use Pandas to scrape the Wiki page and create dataframe

In [2]:
wiki_url="https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695"
df = pd.read_html(wiki_url)[0]
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


Data wrangle to convert dataframe into desired format

In [3]:
# Clean up dataframe

# 1. Rename columns
df.rename(columns={'Postcode':'PostalCode', 'Neighbourhood':'Neighborhood'}, inplace=True)

# 2. Remove rows with unassigned boroughs
df = df[df.Borough != 'Not assigned']


Apply groupby aggregation to aggregate neighborhoods per PostalCode

In [4]:
foo_join = lambda a: ", ".join(a)
new_df = df.groupby(by=['PostalCode', 'Borough'], as_index=False).agg({'Neighborhood':foo_join})
new_df

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


Display shape of dataframe

In [5]:
new_df.shape

(103, 3)

Fetch Latitude Longitude Information from CSV into a new dataframe

In [6]:
ll_info_df = pd.read_csv("https://cocl.us/Geospatial_data")
ll_info_df

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


Rename Postal Code column to match the dataframe with neighborhood info in order to facilitate easy merge

In [7]:
ll_info_df.rename(columns={"Postal Code":"PostalCode"}, inplace=True)
ll_info_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the two dataframes into one

In [8]:
merged_df = pd.merge(left=new_df, right=ll_info_df)
merged_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


# Get Lat & Lng of Toronto

In [9]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Toronto are 43.653963, -79.387207.


# Create a map of Toronto with the Boroughs & Neighborhoods superimposed

In [10]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
#for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
for lat, lng, borough, neighborhood in zip(merged_df['Latitude'], merged_df['Longitude'], merged_df['Borough'], merged_df['Neighborhood']):
    label = '{} : {}'.format(borough, neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Let us analyse some of the boroughs in Toronto

In [11]:
#Find the biggest borough(one that covers most number of postcodes, implying a larger area)
biggest_borough = merged_df['Borough'].value_counts().idxmax()

biggest_borough_data = merged_df[merged_df.Borough == biggest_borough]
biggest_borough_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
17,M2H,North York,Hillcrest Village,43.803762,-79.363452
18,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556
19,M2K,North York,Bayview Village,43.786947,-79.385975
20,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714
21,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493
22,M2N,North York,Willowdale South,43.770120,-79.408493
23,M2P,North York,York Mills West,43.752758,-79.400049
24,M2R,North York,Willowdale West,43.782736,-79.442259
25,M3A,North York,Parkwoods,43.753259,-79.329656
26,M3B,North York,Don Mills North,43.745906,-79.352188


Since North York has the maximum number of rows in our dataframe, this indicates that it covers the most number of postcodes, as compared to other boroughs. This leads us to believe that it could be a geographically large enough borough for us to perform analysis and gain insights from.

In [12]:
address = 'North York, Toronto, Canada'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York, Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York, Toronto are 43.7543263, -79.44911696639593.


In [13]:
# create map of North York using latitude and longitude values
map_northyork = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(biggest_borough_data['Latitude'], biggest_borough_data['Longitude'], biggest_borough_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_northyork) 
    
map_northyork

In [14]:
neigh_lat = merged_df.loc[75, 'Latitude']
neigh_lng = merged_df.loc[75, 'Longitude']
print("Latitude: ", neigh_lat)
print("Longitude: ", neigh_lng)

Latitude:  43.669542
Longitude:  -79.4225637


Setup Foursquare credentials

In [15]:
CLIENT_ID = '0KXFEGYI4ZV1Q1IZZ02C1LZ5RNOYMUFUOL5IFBII4F03HGIL' # your Foursquare ID
CLIENT_SECRET = 'AC0L1B2CSY14CJG4V4F2HV4J5NFFIVMUHFPYFIOK2SNFTUSW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: 0KXFEGYI4ZV1Q1IZZ02C1LZ5RNOYMUFUOL5IFBII4F03HGIL
CLIENT_SECRET:AC0L1B2CSY14CJG4V4F2HV4J5NFFIVMUHFPYFIOK2SNFTUSW


Use the explore API to figure out the different venues listed in the borough of North York

In [16]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neigh_lat, neigh_lng, VERSION, radius, LIMIT)
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e73d3ebb1cac0001bfec9ce'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Christie Pits',
  'headerFullLocation': 'Christie Pits, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 18,
  'suggestedBounds': {'ne': {'lat': 43.6740420045, 'lng': -79.41635411972038},
   'sw': {'lat': 43.6650419955, 'lng': -79.42877328027961}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4adcfd7cf964a5203e6321e3',
       'name': 'Fiesta Farms',
       'location': {'address': '200 Christie St',
        'crossStreet': 'at Essex St',
        'lat': 43.66847077052224,
        'lng': -79.42048512748114,
        'labeledLatLngs': [{'label': 'd

In [17]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [18]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

#Dataframe that contains the different categories of venues listed in the borough
nearby_venues.head()

,name,categories,lat,lng
0,Fiesta Farms,Grocery Store,43.668471,-79.420485
1,Contra Cafe,Café,43.669107,-79.426105
2,Vinny’s Panini,Italian Restaurant,43.670679,-79.426148
3,Starbucks,Coffee Shop,43.671530,-79.421400
4,Scout and Cash Caffe,Café,43.667360,-79.419938


In [19]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [20]:
#Dataframe containing all the listed venues across all neighbourhoods in the borough of North York
biggest_borough_venues = getNearbyVenues(names=biggest_borough_data['Neighborhood'],
                                   latitudes=biggest_borough_data['Latitude'],
                                   longitudes=biggest_borough_data['Longitude']
                                  )



Hillcrest Village
Fairview, Henry Farm, Oriole
Bayview Village
Silver Hills, York Mills
Newtonbrook, Willowdale
Willowdale South
York Mills West
Willowdale West
Parkwoods
Don Mills North
Flemingdon Park, Don Mills South
Bathurst Manor, Downsview North, Wilson Heights
Northwood Park, York University
CFB Toronto, Downsview East
Downsview West
Downsview Central
Downsview Northwest
Victoria Village
Bedford Park, Lawrence Manor East
Lawrence Heights, Lawrence Manor
Glencairn
Downsview, North Park, Upwood Park
Humber Summit
Emery, Humberlea


In [21]:
print(biggest_borough_venues.shape)
biggest_borough_venues[biggest_borough_venues.Neighborhood =='Newtonbrook, Willowdale']

(241, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category


In [22]:
biggest_borough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Bathurst Manor, Downsview North, Wilson Heights",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
"CFB Toronto, Downsview East",3,3,3,3,3,3
Don Mills North,5,5,5,5,5,5
Downsview Central,4,4,4,4,4,4
Downsview Northwest,4,4,4,4,4,4
Downsview West,5,5,5,5,5,5
"Downsview, North Park, Upwood Park",3,3,3,3,3,3


# Analyse each set of neighborhoods

In [23]:
# one hot encoding
biggest_borough_onehot = pd.get_dummies(biggest_borough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
biggest_borough_onehot['Neighborhood'] = biggest_borough_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [biggest_borough_onehot.columns[-1]] + list(biggest_borough_onehot.columns[:-1])
biggest_borough_onehot = biggest_borough_onehot[fixed_columns]

biggest_borough_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Hillcrest Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Fairview, Henry Farm, Oriole",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [24]:
biggest_borough_grouped = biggest_borough_onehot.groupby('Neighborhood').mean().reset_index()
biggest_borough_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor, Downsview North, Wilson Heights",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.100000,0.000000,...,0.050000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.250000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park, Lawrence Manor East",0.000000,0.000000,0.038462,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.038462,0.000000,0.038462,0.000000,0.038462,0.000000,0.000000,0.000000
3,"CFB Toronto, Downsview East",0.000000,0.333333,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Don Mills North,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Downsview Central,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Downsview Northwest,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Downsview West,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.200000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Downsview, North Park, Upwood Park",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.333333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Emery, Humberlea",0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# Print each set of neighborhoods along with 5 most common venues

In [25]:
num_top_venues = 5

for hood in biggest_borough_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = biggest_borough_grouped[biggest_borough_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor, Downsview North, Wilson Heights----
                 venue  freq
0          Coffee Shop  0.10
1                 Bank  0.10
2             Pharmacy  0.05
3  Fried Chicken Joint  0.05
4                Diner  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.12
1      Sandwich Place  0.08
2         Coffee Shop  0.08
3          Restaurant  0.08
4    Greek Restaurant  0.04


----CFB Toronto, Downsview East----
               venue  freq
0               Park  0.33
1  Other Repair Shop  0.33
2            Airport  0.33
3          Juice Bar  0.00
4      Movie Theater  0.00


----Don Mills North----
                  venue  freq
0  Gym / Fitness Center   0.2
1  Caribbean Restaurant   0.2
2                  Café   0.2
3        Bas

In [26]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [27]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = biggest_borough_grouped['Neighborhood']

for ind in np.arange(biggest_borough_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(biggest_borough_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor, Downsview North, Wilson Heights",Coffee Shop,Bank,Shopping Mall,Middle Eastern Restaurant,Convenience Store,Deli / Bodega,Pharmacy,Pizza Place,Ice Cream Shop,Bridal Shop
1,Bayview Village,Japanese Restaurant,Chinese Restaurant,Café,Bank,Women's Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
2,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Restaurant,Butcher,Juice Bar,Café,Comfort Food Restaurant,Fast Food Restaurant,Indian Restaurant
3,"CFB Toronto, Downsview East",Park,Airport,Other Repair Shop,Discount Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
4,Don Mills North,Japanese Restaurant,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Women's Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega


It seems like North York is quite a commercial neighbourhood dominated by restaurants and shops. Let us re-affirm this observation by performing clustering, to see if any outliers come to light.

## Cluster Neighborhoods

In [28]:
# set number of clusters
kclusters = 5

biggest_borough_grouped_clustering = biggest_borough_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(biggest_borough_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 3, 0, 0, 0, 0, 4, 2], dtype=int32)

In [29]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

biggest_borough_merged = biggest_borough_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
biggest_borough_merged = biggest_borough_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

biggest_borough_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,M2H,North York,Hillcrest Village,43.803762,-79.363452,0.0,Golf Course,Mediterranean Restaurant,Pool,Dog Run,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
18,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0.0,Clothing Store,Coffee Shop,Women's Store,Fast Food Restaurant,Bank,Japanese Restaurant,Food Court,Shoe Store,Bakery,Tea Room
19,M2K,North York,Bayview Village,43.786947,-79.385975,0.0,Japanese Restaurant,Chinese Restaurant,Café,Bank,Women's Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
20,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,1.0,Cafeteria,Women's Store,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
21,M2M,North York,"Newtonbrook, Willowdale",43.789053,-79.408493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,M2N,North York,Willowdale South,43.770120,-79.408493,0.0,Ramen Restaurant,Pizza Place,Sandwich Place,Coffee Shop,Café,Sushi Restaurant,Grocery Store,Movie Theater,Bubble Tea Shop,Pet Store
23,M2P,North York,York Mills West,43.752758,-79.400049,0.0,Park,Convenience Store,Flower Shop,Bank,Discount Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
24,M2R,North York,Willowdale West,43.782736,-79.442259,0.0,Grocery Store,Pharmacy,Pizza Place,Bank,Butcher,Coffee Shop,Discount Store,Women's Store,Dim Sum Restaurant,Concert Hall
25,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Park,Food & Drink Shop,Discount Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
26,M3B,North York,Don Mills North,43.745906,-79.352188,0.0,Japanese Restaurant,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Women's Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega


Check to see if there are NaNs in the dataframe

In [30]:
biggest_borough_merged.shape
(pd.isna(biggest_borough_merged)==True).sum().sum()

11

Drop Row containing NaNs - this particular neighborhood does not have any venues listed against it

In [31]:
biggest_borough_merged.dropna(axis=0, inplace=True)
biggest_borough_merged

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,M2H,North York,Hillcrest Village,43.803762,-79.363452,0.0,Golf Course,Mediterranean Restaurant,Pool,Dog Run,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
18,M2J,North York,"Fairview, Henry Farm, Oriole",43.778517,-79.346556,0.0,Clothing Store,Coffee Shop,Women's Store,Fast Food Restaurant,Bank,Japanese Restaurant,Food Court,Shoe Store,Bakery,Tea Room
19,M2K,North York,Bayview Village,43.786947,-79.385975,0.0,Japanese Restaurant,Chinese Restaurant,Café,Bank,Women's Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
20,M2L,North York,"Silver Hills, York Mills",43.757490,-79.374714,1.0,Cafeteria,Women's Store,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant
22,M2N,North York,Willowdale South,43.770120,-79.408493,0.0,Ramen Restaurant,Pizza Place,Sandwich Place,Coffee Shop,Café,Sushi Restaurant,Grocery Store,Movie Theater,Bubble Tea Shop,Pet Store
23,M2P,North York,York Mills West,43.752758,-79.400049,0.0,Park,Convenience Store,Flower Shop,Bank,Discount Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
24,M2R,North York,Willowdale West,43.782736,-79.442259,0.0,Grocery Store,Pharmacy,Pizza Place,Bank,Butcher,Coffee Shop,Discount Store,Women's Store,Dim Sum Restaurant,Concert Hall
25,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Park,Food & Drink Shop,Discount Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
26,M3B,North York,Don Mills North,43.745906,-79.352188,0.0,Japanese Restaurant,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Women's Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
27,M3C,North York,"Flemingdon Park, Don Mills South",43.725900,-79.340923,0.0,Restaurant,Beer Store,Coffee Shop,Asian Restaurant,Gym,Italian Restaurant,Discount Store,Japanese Restaurant,Concert Hall,Bike Shop


# Visualise clusters in a map

In [32]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(biggest_borough_merged['Latitude'], biggest_borough_merged['Longitude'], biggest_borough_merged['Neighborhood'], biggest_borough_merged['Cluster Labels'].astype(int)):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine clusters

# Cluster 1

In [33]:
biggest_borough_merged.loc[biggest_borough_merged['Cluster Labels'] == 0, biggest_borough_merged.columns[[1] + list(range(5, biggest_borough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,North York,0.0,Golf Course,Mediterranean Restaurant,Pool,Dog Run,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
18,North York,0.0,Clothing Store,Coffee Shop,Women's Store,Fast Food Restaurant,Bank,Japanese Restaurant,Food Court,Shoe Store,Bakery,Tea Room
19,North York,0.0,Japanese Restaurant,Chinese Restaurant,Café,Bank,Women's Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
22,North York,0.0,Ramen Restaurant,Pizza Place,Sandwich Place,Coffee Shop,Café,Sushi Restaurant,Grocery Store,Movie Theater,Bubble Tea Shop,Pet Store
23,North York,0.0,Park,Convenience Store,Flower Shop,Bank,Discount Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
24,North York,0.0,Grocery Store,Pharmacy,Pizza Place,Bank,Butcher,Coffee Shop,Discount Store,Women's Store,Dim Sum Restaurant,Concert Hall
26,North York,0.0,Japanese Restaurant,Gym / Fitness Center,Caribbean Restaurant,Café,Baseball Field,Women's Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
27,North York,0.0,Restaurant,Beer Store,Coffee Shop,Asian Restaurant,Gym,Italian Restaurant,Discount Store,Japanese Restaurant,Concert Hall,Bike Shop
28,North York,0.0,Coffee Shop,Bank,Shopping Mall,Middle Eastern Restaurant,Convenience Store,Deli / Bodega,Pharmacy,Pizza Place,Ice Cream Shop,Bridal Shop
29,North York,0.0,Coffee Shop,Miscellaneous Shop,Metro Station,Caribbean Restaurant,Falafel Restaurant,Massage Studio,Bar,Dim Sum Restaurant,Concert Hall,Construction & Landscaping


# Cluster 2

In [34]:
biggest_borough_merged.loc[biggest_borough_merged['Cluster Labels'] == 1, biggest_borough_merged.columns[[1] + list(range(5, biggest_borough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,North York,1.0,Cafeteria,Women's Store,Dog Run,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant


# Cluster 3

In [35]:
biggest_borough_merged.loc[biggest_borough_merged['Cluster Labels'] == 2, biggest_borough_merged.columns[[1] + list(range(5, biggest_borough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
97,North York,2.0,Baseball Field,Women's Store,Electronics Store,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dim Sum Restaurant


# Cluster 4

In [36]:
biggest_borough_merged.loc[biggest_borough_merged['Cluster Labels'] == 3, biggest_borough_merged.columns[[1] + list(range(5, biggest_borough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
25,North York,3.0,Park,Food & Drink Shop,Discount Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
30,North York,3.0,Park,Airport,Other Repair Shop,Discount Store,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop


# Cluster 5

In [37]:
biggest_borough_merged.loc[biggest_borough_merged['Cluster Labels'] == 4, biggest_borough_merged.columns[[1] + list(range(5, biggest_borough_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
79,North York,4.0,Park,Construction & Landscaping,Bakery,Dog Run,Comfort Food Restaurant,Concert Hall,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


The results of our clustering exercise only re-affirm the initial observation that North York is a commercial neighbourhood with restaurants being the most common type of venue. This is demonstrated by the fact that most of the neighbourhoods in the borough were clustered within cluster 1 & 2 - clusters with restaurants as the most common type of venues. Clustering only brought out a handful of neighborhoods (Cluster 3, 4 and 5) that don't have as many restaurants, owing to the fact that probably these are areas where schools/ universities are sitated. These are not necessarily residential clusters, but the presence of the schools/ unversities impact the type of venues in these neighbourhoods.